In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

2024-03-09 00:34:25.275913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.11.0'

In [3]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:376,:T,:]


Y  = [1 for _ in range(376)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(376, 15, 20)
(376, 2)


In [4]:
print(X1.shape)
print(X1[0].shape)
print(X1[0])

BUFFER_SIZE = 1000
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

(376, 15, 20)
(15, 20)
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


2024-03-09 00:34:40.393223: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 00:34:40.395189: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.11.0.


In [6]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [7]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [8]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(25 * 20, activation='sigmoid'))
    model.add(layers.Reshape((25, 20)))
    return model
    

    # model = Sequential([
    #     layers.Input(shape=(latent_dim,)),
    #     layers.Dense(128, activation='relu'),
    #     layers.Dense(256, activation='relu'),
    #     layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
    #     layers.Reshape(output_shape),
    #     layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
    #     layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    # ])

    # return model

In [9]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 1
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator()

noise = tf.random.normal([batch_size, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

<tf.Tensor: shape=(1, 25, 20), dtype=float32, numpy=
array([[[0.5632561 , 0.59868664, 0.5302857 , 0.4743488 , 0.466737  ,
         0.34883004, 0.42490128, 0.45570356, 0.5157759 , 0.43227735,
         0.5345773 , 0.53017   , 0.64453214, 0.60913026, 0.46206757,
         0.59450036, 0.60547906, 0.41075704, 0.4612134 , 0.64958704],
        [0.537805  , 0.6134808 , 0.43480632, 0.45223516, 0.4935239 ,
         0.617118  , 0.5539922 , 0.42951494, 0.4734365 , 0.71595275,
         0.5659221 , 0.48000106, 0.5486674 , 0.3749019 , 0.591867  ,
         0.62894344, 0.45892832, 0.619879  , 0.6292353 , 0.48482794],
        [0.53032124, 0.54135466, 0.3992784 , 0.45691356, 0.42348823,
         0.54346764, 0.40301245, 0.635292  , 0.6087729 , 0.48971877,
         0.47702965, 0.5066979 , 0.5937844 , 0.5010254 , 0.5070725 ,
         0.5717561 , 0.46633738, 0.6082328 , 0.35570124, 0.56770587],
        [0.6369611 , 0.59157073, 0.5317608 , 0.4725489 , 0.49271443,
         0.47108024, 0.59768707, 0.44858932, 0.

In [10]:
discriminator = build_discriminator()
discriminator.load_weights('./acp_mhcnn_weights.h5')

In [11]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 269ms/step


array([[0.30780783, 0.6921922 ],
       [0.17504199, 0.824958  ],
       [0.31832418, 0.68167585],
       [0.18360284, 0.81639713],
       [0.19149974, 0.80850023]], dtype=float32)

In [12]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

1/1 [==============================] - 0s 186ms/step
[[9.9999869e-01 1.3249029e-06]
 [9.9999928e-01 7.0653169e-07]
 [9.9999964e-01 3.3815178e-07]
 [9.9999905e-01 8.9603759e-07]
 [9.9999952e-01 4.7471184e-07]]


In [13]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [14]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

# def generator_loss(disc_output):
#     batch_size = tf.shape(disc_output)[0]
#     num_classes = tf.shape(disc_output)[1]
#     desired_output = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, num_classes - 1))], axis=1)
    
#     # Define binary cross entropy loss
#     bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
#     # Calculate loss
#     loss = bce(desired_output, disc_output)
    
#     return loss

In [15]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [16]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

fake_prediction = discriminator.predict([fake_seqs[:,:15,:]])
real_prediction = discriminator.predict([X1[:5]])
print(fake_prediction)
print(real_prediction)

# disc_loss = discriminator_loss(Y[:5], real_prediction)
gen_loss = generator_loss(fake_prediction)
print(gen_loss)
disc_loss = discriminator_loss(real_prediction, fake_prediction)
print(disc_loss)

1/1 [==============================] - 0s 94ms/step
[[9.9999881e-01 1.1870885e-06]
 [9.9999774e-01 2.2686256e-06]
 [9.9999976e-01 2.6769021e-07]
 [9.9999809e-01 1.9313168e-06]
 [9.9999905e-01 9.4836673e-07]]
[[0.30780783 0.6921922 ]
 [0.17504199 0.824958  ]
 [0.31832418 0.68167585]
 [0.18360284 0.81639713]
 [0.19149974 0.80850023]]
tf.Tensor(6.8207464, shape=(), dtype=float32)
tf.Tensor(7.706738, shape=(), dtype=float32)


In [17]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [18]:
def generate_and_save_images(model, epoch, test_input):
    pass

In [19]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32

EPOCHS = 500
noise_dim = 100

In [22]:
@tf.function
def train_step(peptides):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_bpf = generator(noise, training=True)

      real_output = discriminator(peptides, training=True)
      fake_output = discriminator(generated_bpf[:,:15,:], training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    print(gen_loss)
    print(disc_loss)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(epochs):
  for epoch in range(epochs):
    dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

    start = time.time()

    for seq_batch in dataset:
      train_step(seq_batch)

    # Produce images for the GIF as you go
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [23]:
train(EPOCHS)

Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Time for epoch 1 is 8.748155117034912 sec
Time for epoch 2 is 1.1100022792816162 sec
Time for epoch 3 is 1.315201997756958 sec
Time for epoch 4 is 0.8880801200866699 sec
Time for epoch 5 is 0.7214643955230713 sec
Time for epoch 6 is 0.8460509777069092 sec
Time for epoch 7 is 1.2163808345794678 sec
Time for epoch 8 is 1.182375192642212 sec
Time for epoch 9 is 1.1970548629760742 sec
Time for epoch 10 is 0.9175777435302734 sec
Time for epoch 11 is 1.0940961837768555 sec
Time for epoch 12 is 1.2876603603363037 sec
Time for epoch 13 is 1.2435963153839111 sec
Time for epoch 14 is 1.1411387920379639 sec
Time for epoch 15 is 1.2384791374206543 se

In [36]:
noise = tf.random.normal([5, 100])
#, tf.GradientTape() as disc_tape
peptides = X1[:5, :15, :]
with tf.GradientTape() as gen_tape:
    generated_bpf = generator(noise, training=True)

    # real_output = discriminator(peptides, training=True)
    fake_output = discriminator(generated_bpf[:,:15,:], training=True)

    gen_loss = generator_loss(fake_output)
    # disc_loss = discriminator_loss(real_output, fake_output)
    
    print(gen_loss)
    # print(disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    # gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    print(gradients_of_generator)

tf.Tensor(1.2180667, shape=(), dtype=float32)
[<tf.Tensor: shape=(100, 256), dtype=float32, numpy=
array([[-1.2657368e-02,  6.9917133e-03,  6.0670548e-03, ...,
        -1.6835192e-02, -4.7025052e-03,  2.5047136e-03],
       [ 2.3169793e-02, -4.2005438e-02,  7.3910793e-03, ...,
         8.0881245e-02, -1.2484933e-01,  3.0256495e-02],
       [-1.0271495e-02,  4.4948120e-02,  2.8028844e-03, ...,
        -2.5511788e-02,  4.2048875e-02,  1.0373569e-02],
       ...,
       [ 1.2098706e-02, -7.9551907e-03, -9.0531558e-03, ...,
         5.4907920e-03,  4.2967238e-02, -4.0880926e-03],
       [ 7.1380506e-03,  1.9250286e-04, -2.9733729e-02, ...,
        -6.7231312e-02,  2.4482787e-01, -3.6508486e-02],
       [ 1.7665628e-02, -1.6757061e-03, -1.1706840e-02, ...,
         1.1587529e-02,  5.1152870e-02, -2.5116103e-03]], dtype=float32)>, <tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 3.22254561e-02, -1.07854285e-04,  1.18630454e-02,  1.24601424e-02,
        3.22683118e-02,  2.62602209e-03,

In [50]:
noise = tf.random.normal([100, 100])

generated_bpf = generator(noise, training=True)

fake_output = discriminator(generated_bpf[:,:15,:], training=True)

correct = 0
total = 0
for key, output in enumerate(fake_output):
    if output[1] > 0.25:
        correct += 1
    total += 1

print(correct / total)

0.87
